# Lens Kit Class Scratch 

## Goals:
    Create a Lens Kit Class Object that outputs recommendations needed for RMSE and custom metrics
     
     
     
 

In [12]:
import lenskit.datasets  
import time
from lenskit import batch, topn, util
from lenskit import crossfold as xf
import lenskit.algorithms 
#uncomment below cell
from lenskit.algorithms import Recommender, Predictor, als, item_knn as knn
from lenskit import topn
from  sklearn.metrics import r2_score,mean_squared_error

import numpy as np


In [2]:
import os
os.getcwd()
%cd C:/Users/jonah/School/1004/finalproj


C:\Users\jonah\School\1004\finalproj


In [3]:
#assumes dataset is in the right plaec
ratings = lenskit.datasets.MovieLens('ml-latest-small').ratings
#ratings = lenskit.datasets.MovieLens('ml-latest').ratings

In [71]:
#change partitions to a higher number when working with large 
for i, tp in enumerate(xf.partition_users(ratings, partitions=1, method=xf.LastFrac(.4))):
    print(i)
    train=tp.train
    test=tp.test
    

#create user item average values and scale them by .5
user_means = train.groupby('user').mean().rating*.5
item_means = train.groupby('item').mean().rating*.5

#update ratings column by mapping user id, and subtracting per user half of their mean ratings from all user-item ratings
train['rating'] = train['rating'].sub(train['user'].map(user_means)).fillna(train['rating'])

#update ratings column by mapping movie id and subtrating per movie, half of it's average rating from each uesr-item rating
train['rating'] = train['rating'].sub(train['item'].map(item_means)).fillna(train['rating'])

0


<ipython-input-71-51089efb5dc5>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['rating'] = train['rating'].sub(train['user'].map(user_means)).fillna(train['rating'])
<ipython-input-71-51089efb5dc5>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['rating'] = train['rating'].sub(train['item'].map(item_means)).fillna(train['rating'])


# sanity check, test and val data are after train data

# create val and test data

In [72]:
#val = test.loc[test.user.isin(range(1,int(283228/2)))]

"""
changed the below to reflect how we create validation in Spark
"""

val = test.loc[test['user']%2==0]
print(val.shape)
print(len(set(val["user"])))

(18645, 4)
305


In [73]:
#test = test.loc[test.user.isin(range(int(283228/2),611))]
test = test.loc[test['user']%2==1]
print(test.shape)
print(len(set(test["user"])))

(21670, 4)
305


## Sanity checks test and val data have no user intersection


In [8]:
set(val.user.unique()).intersection(set(test.user.unique()))


set()

In [9]:
model = als.BiasedMF(5, iterations=20, method='lu', progress=None, rng_spec = 5)
print('model')
rec= Recommender.adapt(model)
print('rec.adapt')
rec.fit(train)

users=train.user.unique()
print("users")
b=lenskit.batch.recommend(rec, [1,2,3,4,5], 100)

could not load LIBBLAS: Could not find module 'libblas' (or one of its dependencies). Try using the full path with constructor syntax.


model
rec.adapt


Numba is using threading layer omp - consider TBB
found 1 potential runtime problems - see https://boi.st/lkpy-perf


users


In [135]:
from typing_extensions import ParamSpecArgs
from  sklearn.metrics import r2_score,mean_squared_error
import numpy as np
import time

def param_search(combos_list, method, users, train, val, user_means, item_means, verbose = True):
    """
    combos_list: array of tuples that we have pyspark times and accuracies for
    method: str -either 'lu' or 'cd'
    users: list of unique userids
    train: train dataset
    val: validation or test set
    user_means: 1:1 mapping of .5 *   average movie rating for a user
    item_means: 1:1 mapping of .5 * the average movie rating across all users


    """
    j= 0 
    out_array=list()
    for i in combos_list:
      j+=1
      if j % 10 == 0:
        frac_completed = round(100*j/len(combos_list),3)
        print(f'finished {frac_completed}% of grid search')
        pass

      model = als.BiasedMF(features = i[0], iterations = i[1], method=method, progress=None, rng_spec = 5, reg = i[2])
      rec= Recommender.adapt(model)
      
      #train start time
      train_start = time.time()
      rec.fit(train)
      
      if verbose:
          print(f'trained{i}')
      
      #train end time
      train_end = time.time()
      
      #pred start time
      pred_start = time.time()
      recommendations = lenskit.batch.recommend(rec, users, 100)
      
      if verbose:
          print(f'recommended{i}')
      
      #pred end time
      pred_end=time.time()
      
      #generate metrics
      metrics = accuracy_metrics(recommendations, val, users, train, rec, user_means, item_means)
      
      if verbose:
          print(f'metrics{i}')
        
      train_time = train_end-train_start
      pred_time = pred_end-pred_start

      out_array.append(metrics+(round(train_time,3),round(pred_time,3)))
      metrics = metrics + (round(train_time,3), round(pred_time,3))

      with open('single_machine_small_results.txt', 'a') as output_file:
                  output_file.write(json.dumps(metrics))
                  output_file.write("\n")
           
    
    return out_array

def accuracy_metrics(recommendations, validation, users, train, rec, user_means,item_means):
    """
    outputs custome precision and recall as we have calculated them
    recommendations: DataFrame of recommendations by user
    validation: unaltered validation df
    model, a model object with the params we are plugging in via grid search
    train, the training data, we have to fit the model to the training data to calculate R^2/RMSE
    user_means -> 1:1 mapping of .5 *   average movie rating for a user
    item_means: 1:1 mapping of .5 * the average movie rating across all users

    returns: an array with  custom precision, custom recall, RMSE, R^2 in that order
    """
    
    
    #perform inner join  
    #print(validation.head())
    recommendations = recommendations[["user","item"]]
    inner_df = recommendations.merge(validation[["user","item", "rating"]], on =["user","item"], how="inner")
    
    val_plus = validation[validation["rating"]>2.5][["user","item"]]
    inner_plus = inner_df[inner_df["rating"]>2.5][["user","item"]]
    
    # print(len(val_plus))
    # print(len(inner_plus))
    # print(len(inner_df))
    # print(len(validation))
    #inner_plus=inner_plus[inner_plus["user"]==1]
    
    inner = inner_df.groupby("user").agg(list)
    inner_plus_users = set((inner_plus["user"]))
    val_users=set(validation["user"])
    
    val_plus = val_plus.groupby("user").agg(list)
    inner_plus = inner_plus.groupby("user").agg(list)
    
    #get precision and recall by user
    prec_array = list()
    recall_array = list()
    
    for user in val_users:
        
        if user in inner_plus_users:
            prec = len(inner_plus.loc[user][0])/len(inner.loc[user][0])
            recall = len(inner_plus.loc[user][0])/len(val_plus.loc[user][0])
            prec_array.append(prec)
            recall_array.append(recall)
            
        else:
            recall_array.append(0)
        
    ## start adding in Regression Evaluation Metrics:

    #create predictions for user-item pairs in the validation dataset 
    RMSE_preds = lenskit.batch.predict(rec, validation, njobs = 3) #njobs can be changed, just randomly typed 4 in there
    #remove cold start predictions 

    #denormalize the predictions
    RMSE_preds['prediction'] = RMSE_preds['prediction'].add(RMSE_preds['user'].map(user_means)).fillna(RMSE_preds['prediction'])
    RMSE_preds['prediction'] = RMSE_preds['prediction'].add(RMSE_preds['item'].map(item_means)).fillna(RMSE_preds['prediction'])
    RMSE_preds['prediction'] -=2.5
    
    #subtract 2.5 from the validation dataset for RMSE purposes
    RMSE_preds['rating'] -= 2.5
    RMSE_preds = RMSE_preds.loc[~RMSE_preds['prediction'].isnull()]
    rmse = mean_squared_error(RMSE_preds['rating'], RMSE_preds['prediction'])**.5
    r_sq = r2_score(RMSE_preds['rating'], RMSE_preds['prediction'])
    return (np.mean(prec_array),np.mean(recall_array), rmse, r_sq)

          

In [ ]:
rank_list = [x*5 for x in range(0,51)]

als_data = pd.read_json('results_als.txt', lines=True)
als_data = pd.concat([als_data.drop(['metrics'], axis=1), als_data['metrics'].apply(pd.Series)], axis=1)

#get all non NA l2 penalizers
l2_list = als_data.regParam.loc[~als_data['regParam'].isnull()].unique()
max_iter = [10]

als_grid = list(itertools.product(*[rank_list,max_iter,l2_list]))

users = train.user.unique()


In [136]:
combos_list = [(50,10)]

param_search(combos_list, "lu", users, train, val)

trained(50, 10)
recommended(50, 10)
15243
1248
1425
18645
1
2
metrics(50, 10)


[(0.925255558634828, 0.0924359449098524, 0.374, 3.736)]

In [ ]:
param_search(als_grid, "lu", users, train, val, user_means, item_means, verbose=False)

In [ ]:
a=train[["user","item"]].groupby("user").agg(set)

In [113]:
len(val)

18645

0.8757894736842106